In [48]:
import pandas as pd
import numpy as np
from dotenv import dotenv_values, find_dotenv
from functions import requirements_clean
import os
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '/'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '/'

# 1. Load in Requirements Matrix 

In [49]:
requirements = pd.read_excel(path_rawdata + 'IxI_TR_2017_PRO_Det.xlsx', sheet_name='2017')
requirements = requirements_clean(requirements, wide=True)

In [50]:
requirements

desc_O,Abrasive product manufacturing,Accommodation,"Accounting, tax preparation, bookkeeping, and payroll services",Adhesive manufacturing,"Advertising, public relations, and related services",Air and gas compressor manufacturing,"Air conditioning, refrigeration, and warm air heating equipment manufacturing",Air transportation,Aircraft engine and engine parts manufacturing,Aircraft manufacturing,...,"Watch, clock, and other measuring and controlling device manufacturing",Water transportation,"Water, sewage and other systems",Wet corn milling,Wholesale electronic markets and agents and brokers,Wineries,Wired telecommunications carriers,Wireless telecommunications carriers (except satellite),Wiring device manufacturing,Wood kitchen cabinet and countertop manufacturing
desc_I,,,,,,,,,,,,,,,,,,,,,
Abrasive product manufacturing,1.050968,0.000104,0.000036,0.000313,0.000117,0.003641,0.001652,0.0001,0.000503,0.001321,...,0.000884,0.000466,0.00006,0.000155,0.000132,0.000205,0.000111,0.00021,0.004608,0.001272
Accommodation,0.001625,1.002916,0.008541,0.002069,0.004819,0.003081,0.0029,0.001818,0.003152,0.001891,...,0.001709,0.003885,0.002104,0.00246,0.002504,0.002223,0.002789,0.005152,0.001866,0.004248
"Accounting, tax preparation, bookkeeping, and payroll services",0.004291,0.006341,1.029665,0.004638,0.004743,0.00638,0.006088,0.003808,0.005,0.003399,...,0.002854,0.007627,0.019775,0.007496,0.010872,0.0072,0.007852,0.014223,0.003998,0.00893
Adhesive manufacturing,0.009834,0.000675,0.000125,1.01569,0.000357,0.001036,0.00061,0.000157,0.000192,0.000382,...,0.000548,0.000309,0.000115,0.000951,0.000402,0.000582,0.000166,0.000288,0.000665,0.00895
"Advertising, public relations, and related services",0.003618,0.011064,0.00433,0.004777,1.008599,0.005646,0.005654,0.00414,0.004598,0.003551,...,0.003752,0.015563,0.001708,0.005173,0.014443,0.005694,0.010206,0.012998,0.00376,0.005713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wineries,0.000031,0.002531,0.000194,0.000042,0.00011,0.000057,0.000053,0.001651,0.000059,0.000033,...,0.000027,0.000496,0.000036,0.000066,0.000058,1.067037,0.000041,0.000073,0.000034,0.000064
Wired telecommunications carriers,0.00784,0.005068,0.003881,0.003161,0.005291,0.005727,0.005007,0.00345,0.006966,0.004647,...,0.00224,0.006602,0.007117,0.003668,0.005719,0.003975,1.067228,0.064176,0.002769,0.004821
Wireless telecommunications carriers (except satellite),0.002042,0.00384,0.003834,0.002144,0.004323,0.003229,0.00301,0.003179,0.004243,0.002693,...,0.001764,0.004677,0.00298,0.003186,0.005925,0.003246,0.007299,1.035964,0.002436,0.004095


# 2. Calculate Delta

In [51]:
# Fill null values with 0
requirements = requirements.fillna(value=0)
# Identity - inverse
delta = np.identity(len(requirements)) - np.linalg.inv(requirements)
delta = pd.DataFrame(delta, index=requirements.index, columns=requirements.columns)

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_1588\1942224861.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  requirements = requirements.fillna(value=0)


In [52]:
# Drops the rows and columns with all zeros 
def drop_zero_rows_and_matching_columns(df):
    zero_rows = (df == 0).all(axis=1)
    rows_to_drop = df.index[zero_rows].tolist()
    df_filtered = df.loc[~zero_rows]
    df_filtered = df_filtered.drop(columns=rows_to_drop, errors='ignore')
    return df_filtered, rows_to_drop

# Function to drop the same industries in the requirements table we dropped from delta so that we can use its indexing
def drop_specified_rows_and_columns(df, drop_list):
    df_filtered = df.drop(index=drop_list, columns=drop_list, errors='ignore')
    return df_filtered

delta_dropped, dropped_industries = drop_zero_rows_and_matching_columns(delta)
# Take the inverse of the result 
delta_dropped = np.linalg.inv(delta_dropped)
# Just dropping the same industries in the requirments table so that I can use the column and index titles
requirements_dropped = drop_specified_rows_and_columns(requirements, dropped_industries)

delta_dropped = pd.DataFrame(delta_dropped, index=requirements_dropped.index, columns=requirements_dropped.columns)
delta = delta_dropped

In [53]:
delta

desc_O,Abrasive product manufacturing,Accommodation,"Accounting, tax preparation, bookkeeping, and payroll services",Adhesive manufacturing,"Advertising, public relations, and related services",Air and gas compressor manufacturing,"Air conditioning, refrigeration, and warm air heating equipment manufacturing",Air transportation,Aircraft engine and engine parts manufacturing,Aircraft manufacturing,...,"Watch, clock, and other measuring and controlling device manufacturing",Water transportation,"Water, sewage and other systems",Wet corn milling,Wholesale electronic markets and agents and brokers,Wineries,Wired telecommunications carriers,Wireless telecommunications carriers (except satellite),Wiring device manufacturing,Wood kitchen cabinet and countertop manufacturing
desc_I,,,,,,,,,,,,,,,,,,,,,
Abrasive product manufacturing,15.409528,32.854587,13.220543,-2.518143,15.638550,-23.576022,2.378713,-6.996636,-0.760883,-17.640773,...,4.457947,74.694604,-431.974313,-30.362149,3.100661,3.830749,-3.268773,-2.891936,6.335362,94.784317
Accommodation,-0.589586,-147.687817,-39.031798,20.793383,-17.214428,54.435431,-9.753380,37.909504,1.822630,-4.505268,...,-36.047637,-425.144149,-238.046184,62.601112,-13.121857,-31.882690,25.544925,-17.514260,-8.183674,-196.685806
"Accounting, tax preparation, bookkeeping, and payroll services",-6.692265,366.895275,96.592603,11.556330,91.040152,-91.232663,13.610392,-56.304629,-4.054459,-46.887349,...,32.966818,-129.175281,-2299.064056,-111.316847,6.390804,21.669257,-28.146335,-2.719587,6.768498,194.254116
Adhesive manufacturing,4.469645,-622.316891,-110.381704,51.953930,13.535063,155.603175,-25.887411,148.514341,7.430787,-19.701856,...,-85.179882,-1266.262650,-819.622775,170.547154,-43.290929,-95.595899,60.374420,-32.184184,-58.798997,-517.231072
"Advertising, public relations, and related services",-7.709899,333.458038,77.167999,-9.096026,69.942081,-131.009232,15.651681,-56.393293,-4.574829,-70.522203,...,61.303336,709.069081,-1015.436458,-169.407465,20.160146,48.633753,-32.092369,3.877854,35.134530,537.348657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wineries,-1.799570,-42.440062,-8.917481,6.598404,-9.265477,11.099325,-1.937178,10.934284,0.346801,-7.161370,...,-5.608935,-45.742395,-190.304747,8.906051,-2.138054,8.579031,7.530216,-5.403719,-1.430189,-21.116598
Wired telecommunications carriers,-11.774688,185.639505,54.852791,-8.093314,20.731561,-94.991322,11.127051,-27.332217,-3.612703,-63.730234,...,47.973757,464.282806,-1429.874563,-126.331792,15.224435,36.671096,-3.013508,-21.886148,21.447573,399.000881
Wireless telecommunications carriers (except satellite),10.921766,-176.980240,-53.598033,9.687477,-28.231983,95.784056,-10.274415,22.852158,3.680501,68.044916,...,-42.198933,-496.860750,1788.364639,134.818238,-13.567649,-30.983574,16.304493,30.165178,-26.087760,-390.357585


In [54]:
industries_in_requirments = set(delta.columns)

# 3. Merge concordance 

In [55]:
concordance = pd.read_pickle("B:\OneDrive - DAZ\Desktop\BEAdata - Tony\cleandata\concordance\concordance6_naics6.pkl")

<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:1: SyntaxWarning: invalid escape sequence '\O'
C:\Users\TonyGui\AppData\Local\Temp\ipykernel_1588\740974240.py:1: SyntaxWarning: invalid escape sequence '\O'
  concordance = pd.read_pickle("B:\OneDrive - DAZ\Desktop\BEAdata - Tony\cleandata\concordance\concordance6_naics6.pkl")


In [87]:
# concordance.to_excel("concordance.xlsx")

In [85]:
# Not all industries are in the concordance file we drop the ones that are not here
indsutries_in_concordance = set(concordance["NAICS_desc"])
industires_not_in_concordance = industries_in_requirments - indsutries_in_concordance
# This one doesn't match exactly im dropping it here but I can also just change the title to match exactly
industires_not_in_concordance.update(['Automotive repair and maintenance'])

150

In [57]:
delta = delta.drop(index=industires_not_in_concordance, columns=industires_not_in_concordance, errors='ignore')
# delta.to_excel('delta.xlsx')

In [58]:
concordance

,product,NAICS_desc
4,New domestic autos,Automobile manufacturing
5,New domestic autos,Motor vehicle body manufacturing
6,New foreign autos,Automobile manufacturing
7,New light trucks,Light truck and utility vehicle manufacturing
8,Used autos,Scrap
...,...,...
701,Final consumption expenditures of nonprofit in...,Religious organizations
702,Final consumption expenditures of nonprofit in...,"Grantmaking, giving, and social advocacy organ..."
703,Final consumption expenditures of nonprofit in...,"Civic, social, professional, and similar organ..."
704,Religious organizations' services to households,Religious organizations


In [59]:
delta

desc_O,Abrasive product manufacturing,Accommodation,"Accounting, tax preparation, bookkeeping, and payroll services",Adhesive manufacturing,"Advertising, public relations, and related services","Air conditioning, refrigeration, and warm air heating equipment manufacturing",Air transportation,Aircraft manufacturing,All other chemical product and preparation manufacturing,All other converted paper product manufacturing,...,Veterinary services,Warehousing and storage,Waste management and remediation services,"Watch, clock, and other measuring and controlling device manufacturing",Water transportation,"Water, sewage and other systems",Wet corn milling,Wineries,Wired telecommunications carriers,Wireless telecommunications carriers (except satellite)
desc_I,,,,,,,,,,,,,,,,,,,,,
Abrasive product manufacturing,15.409528,32.854587,13.220543,-2.518143,15.638550,2.378713,-6.996636,-17.640773,-3.976856,-43.974634,...,-464.281786,-4.023930,-1.673608,4.457947,74.694604,-431.974313,-30.362149,3.830749,-3.268773,-2.891936
Accommodation,-0.589586,-147.687817,-39.031798,20.793383,-17.214428,-9.753380,37.909504,-4.505268,7.229567,118.354766,...,1164.492064,3.708430,2.702434,-36.047637,-425.144149,-238.046184,62.601112,-31.882690,25.544925,-17.514260
"Accounting, tax preparation, bookkeeping, and payroll services",-6.692265,366.895275,96.592603,11.556330,91.040152,13.610392,-56.304629,-46.887349,-1.578480,-148.138775,...,-1600.343217,-20.148453,4.928607,32.966818,-129.175281,-2299.064056,-111.316847,21.669257,-28.146335,-2.719587
Adhesive manufacturing,4.469645,-622.316891,-110.381704,51.953930,13.535063,-25.887411,148.514341,-19.701856,10.063609,354.601137,...,2906.828483,14.199346,11.552872,-85.179882,-1266.262650,-819.622775,170.547154,-95.595899,60.374420,-32.184184
"Advertising, public relations, and related services",-7.709899,333.458038,77.167999,-9.096026,69.942081,15.651681,-56.393293,-70.522203,-15.598145,-267.835304,...,-2879.772295,-20.686094,-2.047382,61.303336,709.069081,-1015.436458,-169.407465,48.633753,-32.092369,3.877854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Water, sewage and other systems",0.767655,239.346523,73.693744,-44.060136,42.386119,10.447735,-39.552617,-9.023786,-8.931302,-217.823220,...,-2179.333277,-9.870556,-3.391613,42.721496,509.380314,1250.292051,-113.113454,48.125687,-39.140236,16.386177
Wet corn milling,-4.627393,116.478369,55.788938,5.654543,38.701275,13.694507,-26.787077,-42.421460,-18.328122,-212.780235,...,-2094.896709,-13.445874,-9.131098,45.704373,899.071206,204.543198,-134.368337,47.700743,-23.302397,1.633515
Wineries,-1.799570,-42.440062,-8.917481,6.598404,-9.265477,-1.937178,10.934284,-7.161370,1.103953,26.466599,...,235.523269,1.716920,-0.470471,-5.608935,-45.742395,-190.304747,8.906051,8.579031,7.530216,-5.403719


# These two for loops merge the corcordance file onto delta (ie delta is now products and not industries) while considering 
# the fact that an industry can be in multiple products. The values in delta are simply divided evenly

In [60]:
# concordance['In#Products'] = concordance['NAICS_desc'].map(concordance['NAICS_desc'].value_counts())
# concordance
# indsutry_to_product = concordance.set_index('NAICS_desc')['product'].to_dict()
# delta_products = delta.rename(columns=indsutry_to_product)

concordance_filtered = concordance[concordance['NAICS_desc'].isin(delta.index)]
# Counts the number of products and industry belongs to 
in_num_products = concordance_filtered['NAICS_desc'].value_counts()
delta_product_columns = pd.DataFrame(index=delta.index, columns=concordance_filtered['product'])

# These two for loops merge the corcordance file onto delta (ie delta is now products and not industries) while considering 
# the fact that an industry can be in multiple products. The values in delta are simply divided evenly
for column in delta_product_columns: 
    industry = concordance_filtered.loc[concordance_filtered['product'] == column, 'NAICS_desc'].values
    delta_product_columns[column] = delta[industry] / in_num_products[industry]

delta_product_cr = pd.DataFrame(columns=delta_product_columns.columns)
lst_for_industry = []
for row in delta_product_columns.index:
    final_value = delta_product_columns.loc[row] / in_num_products[row]
    prodcuts = concordance_filtered[concordance_filtered['NAICS_desc'] == row]['product'].tolist()
    for product in prodcuts: 
        lst_for_industry.append(row)
        final_value_row = pd.DataFrame(final_value).T
        final_value_row.index = [product]
        delta_product_cr = pd.concat([delta_product_cr, final_value_row])


industry = pd.DataFrame(lst_for_industry, columns=['industry'])
industry.index = delta_product_cr.index
delta_product_cr = pd.concat([delta_product_cr, industry], axis = 1)
# Final Delta but columns are rows are now both at the products level
delta_product_cr

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_1588\1356607193.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  delta_product_cr = pd.concat([delta_product_cr, final_value_row])


,New domestic autos,New domestic autos,New foreign autos,New light trucks,Tires,Accessories and parts,Accessories and parts,Accessories and parts,Accessories and parts,Accessories and parts,...,Final consumption expenditures of nonprofit institutions serving households (NPISH),Final consumption expenditures of nonprofit institutions serving households (NPISH),Final consumption expenditures of nonprofit institutions serving households (NPISH),Final consumption expenditures of nonprofit institutions serving households (NPISH),Final consumption expenditures of nonprofit institutions serving households (NPISH),Final consumption expenditures of nonprofit institutions serving households (NPISH),Final consumption expenditures of nonprofit institutions serving households (NPISH),Religious organizations' services to households,Foundations and grantmaking and giving services to households,industry
"Tools, hardware, and supplies",-1005.649161,13.330747,-1005.649161,39450.239138,-0.434612,-0.742256,-1.009075,-0.760743,0.641897,-0.867699,...,-0.090946,0.394022,1321.298810,0.402022,-1.463723,26.898411,2.941676,-1.463723,26.898411,Abrasive product manufacturing
Group housing,287.167032,-6.646690,287.167032,-14452.508083,0.051407,0.442014,0.353309,0.004675,-0.582053,0.390069,...,0.083379,-0.059745,48.626792,-0.073649,0.393811,-14.005095,-2.462879,0.393811,-14.005095,Accommodation
"Amusement parks, campgrounds, and related recreational services",287.167032,-6.646690,287.167032,-14452.508083,0.051407,0.442014,0.353309,0.004675,-0.582053,0.390069,...,0.083379,-0.059745,48.626792,-0.073649,0.393811,-14.005095,-2.462879,0.393811,-14.005095,Accommodation
Hotels and motels,287.167032,-6.646690,287.167032,-14452.508083,0.051407,0.442014,0.353309,0.004675,-0.582053,0.390069,...,0.083379,-0.059745,48.626792,-0.073649,0.393811,-14.005095,-2.462879,0.393811,-14.005095,Accommodation
Housing at schools,287.167032,-6.646690,287.167032,-14452.508083,0.051407,0.442014,0.353309,0.004675,-0.582053,0.390069,...,0.083379,-0.059745,48.626792,-0.073649,0.393811,-14.005095,-2.462879,0.393811,-14.005095,Accommodation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Cable, satellite, and other live television services",-818.453121,-5.146275,-818.453121,37299.962272,-0.313586,-1.029809,-1.574206,-0.495528,0.932927,-1.056442,...,-0.115954,0.339013,-17671.680129,0.381237,-1.893286,37.006916,5.130278,-1.893286,37.006916,Wired telecommunications carriers
"Land-line telephone services, local charges",-818.453121,-5.146275,-818.453121,37299.962272,-0.313586,-1.029809,-1.574206,-0.495528,0.932927,-1.056442,...,-0.115954,0.339013,-17671.680129,0.381237,-1.893286,37.006916,5.130278,-1.893286,37.006916,Wired telecommunications carriers
"Land-line telephone services, long-distance charges",-818.453121,-5.146275,-818.453121,37299.962272,-0.313586,-1.029809,-1.574206,-0.495528,0.932927,-1.056442,...,-0.115954,0.339013,-17671.680129,0.381237,-1.893286,37.006916,5.130278,-1.893286,37.006916,Wired telecommunications carriers
Internet access,-818.453121,-5.146275,-818.453121,37299.962272,-0.313586,-1.029809,-1.574206,-0.495528,0.932927,-1.056442,...,-0.115954,0.339013,-17671.680129,0.381237,-1.893286,37.006916,5.130278,-1.893286,37.006916,Wired telecommunications carriers


# 4. Collapse columns

In [61]:
# Sum columns by products
delta_products = delta_product_cr.groupby(delta_product_cr.columns, axis=1).sum()
delta_products

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_1588\2450155567.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  delta_products = delta_product_cr.groupby(delta_product_cr.columns, axis=1).sum()


,Accessories and parts,Air transportation,Alcohol in purchased meals,All other professional medical services,"Amusement parks, campgrounds, and related recreational services","Audio discs, tapes, vinyl, and permanent digital downloads",Audio equipment,Auto leasing,Bakery products,Beef and veal,...,Veterinary and other services for pets,Video and audio streaming and rental,"Video discs, tapes, and permanent digital downloads",Watches,Water supply and sewage maintenance,Water transportation,Window coverings,Wine,Women's and girls' clothing,industry
"Tools, hardware, and supplies",66.516248,-1.166106,-21.386616,4.417978,1185.893142,-2.814494,15.461432,11.536638,-58.544383,-0.514305,...,-468.455129,6.768662,-0.262626,1.485982,-217.312818,24.898201,-4.324460,3.830749,-5.926450,Abrasive product manufacturing
Group housing,-24.203488,0.902607,11.330309,-1.195975,193.896782,0.437928,-3.331931,-5.907297,23.979029,0.318918,...,167.491847,-2.005003,-0.982844,-1.716554,-17.341601,-20.244959,6.026165,-4.554670,3.731254,Accommodation
"Amusement parks, campgrounds, and related recreational services",-24.203488,0.902607,11.330309,-1.195975,193.896782,0.437928,-3.331931,-5.907297,23.979029,0.318918,...,167.491847,-2.005003,-0.982844,-1.716554,-17.341601,-20.244959,6.026165,-4.554670,3.731254,Accommodation
Hotels and motels,-24.203488,0.902607,11.330309,-1.195975,193.896782,0.437928,-3.331931,-5.907297,23.979029,0.318918,...,167.491847,-2.005003,-0.982844,-1.716554,-17.341601,-20.244959,6.026165,-4.554670,3.731254,Accommodation
Housing at schools,-24.203488,0.902607,11.330309,-1.195975,193.896782,0.437928,-3.331931,-5.907297,23.979029,0.318918,...,167.491847,-2.005003,-0.982844,-1.716554,-17.341601,-20.244959,6.026165,-4.554670,3.731254,Accommodation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Cable, satellite, and other live television services",46.600862,-1.138842,-25.563260,4.199782,-17754.655550,-2.560783,13.750476,7.991423,-60.442025,-0.548871,...,-477.499005,5.467647,0.412262,3.997813,-179.271907,38.690234,-15.967555,9.167774,-7.470261,Wired telecommunications carriers
"Land-line telephone services, local charges",46.600862,-1.138842,-25.563260,4.199782,-17754.655550,-2.560783,13.750476,7.991423,-60.442025,-0.548871,...,-477.499005,5.467647,0.412262,3.997813,-179.271907,38.690234,-15.967555,9.167774,-7.470261,Wired telecommunications carriers
"Land-line telephone services, long-distance charges",46.600862,-1.138842,-25.563260,4.199782,-17754.655550,-2.560783,13.750476,7.991423,-60.442025,-0.548871,...,-477.499005,5.467647,0.412262,3.997813,-179.271907,38.690234,-15.967555,9.167774,-7.470261,Wired telecommunications carriers
Internet access,46.600862,-1.138842,-25.563260,4.199782,-17754.655550,-2.560783,13.750476,7.991423,-60.442025,-0.548871,...,-477.499005,5.467647,0.412262,3.997813,-179.271907,38.690234,-15.967555,9.167774,-7.470261,Wired telecommunications carriers


# 5. Make Matrix to Sales

In [62]:
# Manually clean the make matrix
make_matrix = pd.read_excel("B:/OneDrive - DAZ/Desktop/BEAdata - Tony/rawdata/Supply_2017_DET.xlsx", sheet_name="2017")
make_matrix = make_matrix.iloc[3:,1:]
make_matrix = make_matrix.drop(make_matrix.index[1])
make_matrix.columns = make_matrix.iloc[0]
make_matrix = make_matrix.drop(make_matrix.index[0])
make_matrix.reset_index(drop=True, inplace=True)
make_matrix = make_matrix[[make_matrix.columns[0], 'Total Commodity Output']]
make_matrix.rename(columns={make_matrix.columns[0]: 'Industries', 'Total Commodity Output': "Sales"}, inplace=True)
make_matrix = make_matrix.fillna(value=0)
# Sums are already calculated in the given excel sheet
sales = make_matrix.iloc[:-3]

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_1588\2776680790.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  make_matrix = make_matrix.fillna(value=0)


In [63]:
sales

3,Industries,Sales
0,Oilseed farming,37922
1,Grain farming,66790
2,Vegetable and melon farming,18582
3,Fruit and tree nut farming,29731
4,"Greenhouse, nursery, and floriculture production",19868
...,...,...
397,Other state and local government enterprises,107059
398,Scrap,10763
399,Used and secondhand goods,0
400,Noncomparable imports,0


# 6. Merge concordance with sales 

In [64]:
product_sales = pd.merge(concordance, sales, left_on='NAICS_desc', right_on='Industries')
product_sales = product_sales[["product", "Industries", 'Sales']]
product_sales

,product,Industries,Sales
0,New domestic autos,Automobile manufacturing,32758
1,New domestic autos,Motor vehicle body manufacturing,10556
2,New foreign autos,Automobile manufacturing,32758
3,New light trucks,Light truck and utility vehicle manufacturing,215421
4,Used autos,Scrap,10763
...,...,...,...
684,Final consumption expenditures of nonprofit in...,Religious organizations,100202
685,Final consumption expenditures of nonprofit in...,"Grantmaking, giving, and social advocacy organ...",58458
686,Final consumption expenditures of nonprofit in...,"Civic, social, professional, and similar organ...",95191
687,Religious organizations' services to households,Religious organizations,100202


# 7. Group sum of Sales for every Product

In [65]:
# calculates the number of product categories an Industry belongs to 
product_sales['In#Products'] = product_sales['Industries'].map(product_sales['Industries'].value_counts())
product_sales["Ratio_in_Product"] = product_sales['Sales'] / product_sales['In#Products'] 
product_sales['Sales_Sum'] = product_sales.groupby('product')['Ratio_in_Product'].transform('sum')


# 8. Sales Share 

In [66]:
# Like in delta for sale share I take into account how many products and industry belongs to and use an even ratio to find sale share 
# the sale shares still add up to one
product_sales['Sale Share'] = product_sales["Ratio_in_Product"] / product_sales["Sales_Sum"]
product_sales = product_sales.rename(columns={'Industries': 'industry'})
product_sales

,product,industry,Sales,In#Products,Ratio_in_Product,Sales_Sum,Sale Share
0,New domestic autos,Automobile manufacturing,32758,2,16379.000000,2.693500e+04,0.608094
1,New domestic autos,Motor vehicle body manufacturing,10556,1,10556.000000,2.693500e+04,0.391906
2,New foreign autos,Automobile manufacturing,32758,2,16379.000000,1.637900e+04,1.000000
3,New light trucks,Light truck and utility vehicle manufacturing,215421,1,215421.000000,2.154210e+05,1.000000
4,Used autos,Scrap,10763,6,1793.833333,1.793833e+03,1.000000
...,...,...,...,...,...,...,...
684,Final consumption expenditures of nonprofit in...,Religious organizations,100202,2,50101.000000,1.554667e+06,0.032226
685,Final consumption expenditures of nonprofit in...,"Grantmaking, giving, and social advocacy organ...",58458,3,19486.000000,1.554667e+06,0.012534
686,Final consumption expenditures of nonprofit in...,"Civic, social, professional, and similar organ...",95191,4,23797.750000,1.554667e+06,0.015307
687,Religious organizations' services to households,Religious organizations,100202,2,50101.000000,5.010100e+04,1.000000


# 9. Merge Sale shares and Delta

In [67]:
df_reset = delta_products.reset_index()
delta_products = df_reset.rename(columns={'index': 'product'})

In [68]:
# Because the same products appear more than once, we merge on industry and product 
delta_products_saleshare = delta_products.merge(product_sales, how = 'inner', on = ['industry', 'product'])

In [69]:
delta_products_saleshare

,product,Accessories and parts,Air transportation,Alcohol in purchased meals,All other professional medical services,"Amusement parks, campgrounds, and related recreational services","Audio discs, tapes, vinyl, and permanent digital downloads",Audio equipment,Auto leasing,Bakery products,...,Water transportation,Window coverings,Wine,Women's and girls' clothing,industry,Sales,In#Products,Ratio_in_Product,Sales_Sum,Sale Share
0,"Tools, hardware, and supplies",66.516248,-1.166106,-21.386616,4.417978,1185.893142,-2.814494,15.461432,11.536638,-58.544383,...,24.898201,-4.324460,3.830749,-5.926450,Abrasive product manufacturing,4724,1,4724.000000,130412.799451,0.036223
1,Group housing,-24.203488,0.902607,11.330309,-1.195975,193.896782,0.437928,-3.331931,-5.907297,23.979029,...,-20.244959,6.026165,-4.554670,3.731254,Accommodation,218625,7,31232.142857,31232.142857,1.000000
2,"Amusement parks, campgrounds, and related recr...",-24.203488,0.902607,11.330309,-1.195975,193.896782,0.437928,-3.331931,-5.907297,23.979029,...,-20.244959,6.026165,-4.554670,3.731254,Accommodation,218625,7,31232.142857,170196.942857,0.183506
3,Hotels and motels,-24.203488,0.902607,11.330309,-1.195975,193.896782,0.437928,-3.331931,-5.907297,23.979029,...,-20.244959,6.026165,-4.554670,3.731254,Accommodation,218625,7,31232.142857,31232.142857,1.000000
4,Housing at schools,-24.203488,0.902607,11.330309,-1.195975,193.896782,0.437928,-3.331931,-5.907297,23.979029,...,-20.244959,6.026165,-4.554670,3.731254,Accommodation,218625,7,31232.142857,31232.142857,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,"Cable, satellite, and other live television se...",46.600862,-1.138842,-25.563260,4.199782,-17754.655550,-2.560783,13.750476,7.991423,-60.442025,...,38.690234,-15.967555,9.167774,-7.470261,Wired telecommunications carriers,351841,4,87960.250000,87960.250000,1.000000
621,"Land-line telephone services, local charges",46.600862,-1.138842,-25.563260,4.199782,-17754.655550,-2.560783,13.750476,7.991423,-60.442025,...,38.690234,-15.967555,9.167774,-7.470261,Wired telecommunications carriers,351841,4,87960.250000,152728.059524,0.575927
622,"Land-line telephone services, long-distance ch...",46.600862,-1.138842,-25.563260,4.199782,-17754.655550,-2.560783,13.750476,7.991423,-60.442025,...,38.690234,-15.967555,9.167774,-7.470261,Wired telecommunications carriers,351841,4,87960.250000,95354.500000,0.922455
623,Internet access,46.600862,-1.138842,-25.563260,4.199782,-17754.655550,-2.560783,13.750476,7.991423,-60.442025,...,38.690234,-15.967555,9.167774,-7.470261,Wired telecommunications carriers,351841,4,87960.250000,95354.500000,0.922455


# 10. weightTimesDeltaValue + 11. I Haven't Fixed this Yet Working on it

In [70]:
# I am creating weightTimesDeltaValue by multiplying slae share with the delta value of the product cell 
def multiply_product_and_sale(row):
    product_value = row[row['product']]
    return product_value * row['Sale Share']

delta_products_saleshare['weightTimesDeltaValue'] = delta_products_saleshare.apply(multiply_product_and_sale, axis=1)

In [71]:
delta_products_saleshare = delta_products_saleshare.groupby('product').sum()

In [72]:
delta_products_saleshare

,Accessories and parts,Air transportation,Alcohol in purchased meals,All other professional medical services,"Amusement parks, campgrounds, and related recreational services","Audio discs, tapes, vinyl, and permanent digital downloads",Audio equipment,Auto leasing,Bakery products,Beef and veal,...,Window coverings,Wine,Women's and girls' clothing,industry,Sales,In#Products,Ratio_in_Product,Sales_Sum,Sale Share,weightTimesDeltaValue
product,,,,,,,,,,,,,,,,,,,,,
Accessories and parts,-37.275777,6.430804,49.573320,-2.539059,127187.102083,-3.724002,35.780726,-48.076721,143.122399,2.663854,...,6.692726,-19.061742,16.484862,All other miscellaneous manufacturingBroadcast...,348472,62,205787.416667,3.528881e+06,0.991358,-33.178026
Air transportation,34.215246,-0.307089,-10.121555,1.535788,-3102.592786,-1.339735,8.242077,5.816784,-23.997906,-0.074141,...,-9.401662,3.958952,-4.465529,Air transportation,209931,6,34988.500000,3.498850e+04,1.000000,-0.307089
Alcohol in purchased meals,92.838859,-1.985454,-51.627264,9.783079,-106364.718937,-4.899060,18.207822,18.202361,-115.710223,-1.120180,...,-23.376750,12.544728,-12.866104,Air transportationAll other food and drinking ...,1296986,33,418789.916667,3.350319e+06,1.000000,-14.462765
All other professional medical services,-70.664416,2.562421,41.355408,-6.658129,27593.736390,2.363343,-18.105379,-15.417372,86.651995,0.899596,...,24.315067,-14.996575,12.267910,General and consumer goods rentalOther ambulat...,231716,16,56853.583333,1.705608e+05,1.000000,-0.483040
"Amusement parks, campgrounds, and related recreational services",-526.151635,11.715045,247.527298,-40.278951,26304.354102,17.847107,-184.008634,-92.334147,348.551754,1.352669,...,207.801598,-93.867708,88.457627,AccommodationAmusement parks and arcadesCommer...,589340,40,159594.942857,1.531772e+06,0.937707,-4107.166558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Water supply and sewage maintenance,108.322358,-3.218895,-63.309553,8.683784,-95044.333818,-2.727791,20.720820,29.881064,-174.524859,-1.891495,...,-33.162022,23.529693,-18.925426,Other state and local government enterprisesWa...,187320,5,75816.833333,1.516337e+05,1.000000,166.019178
Water transportation,-18.257022,0.609486,10.308925,-1.458653,20370.931411,0.848065,-4.219708,-4.140408,24.902365,0.238942,...,6.228007,-2.651932,3.112905,Water transportation,52852,3,17617.333333,6.948567e+04,0.253539,-4.491795
Window coverings,6.588332,-0.285986,-1.103787,-0.274349,-5867.803886,0.226762,-3.074148,3.152707,-9.733466,-0.186357,...,5.215105,-0.410059,-0.854366,Curtain and linen millsOther furniture related...,23876,20,5098.192308,1.529458e+04,1.000000,1.778200


In [73]:
# Drop some now unneeded columns
delta_products_saleshare = delta_products_saleshare.drop(columns=['Sales_Sum', 'Sale Share', "Ratio_in_Product", "In#Products", "industry", "Sales"])


In [74]:
delta_products_saleshare

,Accessories and parts,Air transportation,Alcohol in purchased meals,All other professional medical services,"Amusement parks, campgrounds, and related recreational services","Audio discs, tapes, vinyl, and permanent digital downloads",Audio equipment,Auto leasing,Bakery products,Beef and veal,...,Veterinary and other services for pets,Video and audio streaming and rental,"Video discs, tapes, and permanent digital downloads",Watches,Water supply and sewage maintenance,Water transportation,Window coverings,Wine,Women's and girls' clothing,weightTimesDeltaValue
product,,,,,,,,,,,,,,,,,,,,,
Accessories and parts,-37.275777,6.430804,49.573320,-2.539059,127187.102083,-3.724002,35.780726,-48.076721,143.122399,2.663854,...,573.949043,-7.552405,-10.509130,-4.150167,-789.972049,-81.526452,6.692726,-19.061742,16.484862,-33.178026
Air transportation,34.215246,-0.307089,-10.121555,1.535788,-3102.592786,-1.339735,8.242077,5.816784,-23.997906,-0.074141,...,-248.511103,2.478881,0.277598,2.089687,-70.195317,32.152892,-9.401662,3.958952,-4.465529,-0.307089
Alcohol in purchased meals,92.838859,-1.985454,-51.627264,9.783079,-106364.718937,-4.899060,18.207822,18.202361,-115.710223,-1.120180,...,-959.153152,10.732673,0.752641,5.141880,-167.312502,80.701303,-23.376750,12.544728,-12.866104,-14.462765
All other professional medical services,-70.664416,2.562421,41.355408,-6.658129,27593.736390,2.363343,-18.105379,-15.417372,86.651995,0.899596,...,672.538068,-8.562216,-2.195853,-5.795550,93.568662,-64.850038,24.315067,-14.996575,12.267910,-0.483040
"Amusement parks, campgrounds, and related recreational services",-526.151635,11.715045,247.527298,-40.278951,26304.354102,17.847107,-184.008634,-92.334147,348.551754,1.352669,...,4963.439526,-63.763870,-10.718443,-50.125650,1834.191136,-635.542916,207.801598,-93.867708,88.457627,-4107.166558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Water supply and sewage maintenance,108.322358,-3.218895,-63.309553,8.683784,-95044.333818,-2.727791,20.720820,29.881064,-174.524859,-1.891495,...,-1081.974735,13.056657,5.010737,6.926131,313.641245,83.107231,-33.162022,23.529693,-18.925426,166.019178
Water transportation,-18.257022,0.609486,10.308925,-1.458653,20370.931411,0.848065,-4.219708,-4.140408,24.902365,0.238942,...,208.883862,-1.596044,-0.351092,-1.264813,71.326840,-17.716378,6.228007,-2.651932,3.112905,-4.491795
Window coverings,6.588332,-0.285986,-1.103787,-0.274349,-5867.803886,0.226762,-3.074148,3.152707,-9.733466,-0.186357,...,-5.857977,0.041022,0.601414,-0.121359,36.242141,3.583006,5.215105,-0.410059,-0.854366,1.778200


# 12. Calculate Gamma

In [75]:
# Ignore weightTimesDeltaValue then finding row sum 
delta_columns_wanted = delta_products_saleshare.iloc[:, :-1]
row_sums = delta_columns_wanted.sum(axis=1)
gamma = pd.DataFrame(row_sums, columns=['Row_Sum'])
gamma.index = delta_products_saleshare.index 

In [76]:
gamma

,Row_Sum
product,
Accessories and parts,2.985967e+07
Air transportation,-2.031340e+07
Alcohol in purchased meals,-7.391721e+07
All other professional medical services,5.104492e+07
"Amusement parks, campgrounds, and related recreational services",4.141118e+08
...,...
Water supply and sewage maintenance,-7.950226e+07
Water transportation,1.629299e+07
Window coverings,8.782768e+05


In [77]:
# Merge Delta with gamma 
delta_products_saleshare = delta_products_saleshare.merge(gamma, left_on=gamma.index, right_on=delta_products_saleshare.index)

In [78]:
delta_products_saleshare

,key_0,Accessories and parts,Air transportation,Alcohol in purchased meals,All other professional medical services,"Amusement parks, campgrounds, and related recreational services","Audio discs, tapes, vinyl, and permanent digital downloads",Audio equipment,Auto leasing,Bakery products,...,Video and audio streaming and rental,"Video discs, tapes, and permanent digital downloads",Watches,Water supply and sewage maintenance,Water transportation,Window coverings,Wine,Women's and girls' clothing,weightTimesDeltaValue,Row_Sum
0,Accessories and parts,-37.275777,6.430804,49.573320,-2.539059,127187.102083,-3.724002,35.780726,-48.076721,143.122399,...,-7.552405,-10.509130,-4.150167,-789.972049,-81.526452,6.692726,-19.061742,16.484862,-33.178026,2.985967e+07
1,Air transportation,34.215246,-0.307089,-10.121555,1.535788,-3102.592786,-1.339735,8.242077,5.816784,-23.997906,...,2.478881,0.277598,2.089687,-70.195317,32.152892,-9.401662,3.958952,-4.465529,-0.307089,-2.031340e+07
2,Alcohol in purchased meals,92.838859,-1.985454,-51.627264,9.783079,-106364.718937,-4.899060,18.207822,18.202361,-115.710223,...,10.732673,0.752641,5.141880,-167.312502,80.701303,-23.376750,12.544728,-12.866104,-14.462765,-7.391721e+07
3,All other professional medical services,-70.664416,2.562421,41.355408,-6.658129,27593.736390,2.363343,-18.105379,-15.417372,86.651995,...,-8.562216,-2.195853,-5.795550,93.568662,-64.850038,24.315067,-14.996575,12.267910,-0.483040,5.104492e+07
4,"Amusement parks, campgrounds, and related recr...",-526.151635,11.715045,247.527298,-40.278951,26304.354102,17.847107,-184.008634,-92.334147,348.551754,...,-63.763870,-10.718443,-50.125650,1834.191136,-635.542916,207.801598,-93.867708,88.457627,-4107.166558,4.141118e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Water supply and sewage maintenance,108.322358,-3.218895,-63.309553,8.683784,-95044.333818,-2.727791,20.720820,29.881064,-174.524859,...,13.056657,5.010737,6.926131,313.641245,83.107231,-33.162022,23.529693,-18.925426,166.019178,-7.950226e+07
189,Water transportation,-18.257022,0.609486,10.308925,-1.458653,20370.931411,0.848065,-4.219708,-4.140408,24.902365,...,-1.596044,-0.351092,-1.264813,71.326840,-17.716378,6.228007,-2.651932,3.112905,-4.491795,1.629299e+07
190,Window coverings,6.588332,-0.285986,-1.103787,-0.274349,-5867.803886,0.226762,-3.074148,3.152707,-9.733466,...,0.041022,0.601414,-0.121359,36.242141,3.583006,5.215105,-0.410059,-0.854366,1.778200,8.782768e+05
191,Wine,-33.849605,1.822381,18.949520,-0.984592,15663.470710,0.411542,-2.344689,-9.952827,54.417240,...,-2.308654,-1.722893,-1.869645,-96.391606,-15.247465,5.726997,8.579031,4.965338,8.579031,1.361545e+07


# 13. Calculate Omega

In [79]:
delta_products_saleshare.set_index(delta_products_saleshare.columns[0], inplace=True)
delta_divide_columns = delta_products_saleshare.columns.difference(['weightTimesDeltaValue', 'Row_Sum'])
delta_products_saleshare[delta_divide_columns] = delta_products_saleshare[delta_divide_columns].div(delta_products_saleshare['Row_Sum'], axis=0)

In [80]:
delta_products_saleshare_last2 = delta_products_saleshare.columns[-2:]
weight_and_row_sum = delta_products_saleshare[delta_products_saleshare_last2].copy()
delta_products_saleshare.drop(columns=delta_products_saleshare_last2, inplace=True)

In [81]:
delta_products_saleshare

,Accessories and parts,Air transportation,Alcohol in purchased meals,All other professional medical services,"Amusement parks, campgrounds, and related recreational services","Audio discs, tapes, vinyl, and permanent digital downloads",Audio equipment,Auto leasing,Bakery products,Beef and veal,...,Vegetables (fresh),Veterinary and other services for pets,Video and audio streaming and rental,"Video discs, tapes, and permanent digital downloads",Watches,Water supply and sewage maintenance,Water transportation,Window coverings,Wine,Women's and girls' clothing
key_0,,,,,,,,,,,,,,,,,,,,,
Accessories and parts,-0.000001,2.153676e-07,1.660210e-06,-8.503305e-08,0.004259,-1.247168e-07,1.198296e-06,-1.610089e-06,4.793168e-06,8.921244e-08,...,4.821752e-07,0.000019,-2.529300e-07,-3.519507e-07,-1.389891e-07,-0.000026,-0.000003,2.241393e-07,-6.383776e-07,5.520779e-07
Air transportation,-0.000002,1.511755e-08,4.982699e-07,-7.560467e-08,0.000153,6.595329e-08,-4.057459e-07,-2.863521e-07,1.181383e-06,3.649874e-09,...,1.021782e-07,0.000012,-1.220318e-07,-1.366574e-08,-1.028724e-07,0.000003,-0.000002,4.628306e-07,-1.948936e-07,2.198317e-07
Alcohol in purchased meals,-0.000001,2.686052e-08,6.984471e-07,-1.323518e-07,0.001439,6.627767e-08,-2.463272e-07,-2.462533e-07,1.565403e-06,1.515452e-08,...,1.357367e-07,0.000013,-1.451986e-07,-1.018222e-08,-6.956269e-08,0.000002,-0.000001,3.162558e-07,-1.697132e-07,1.740610e-07
All other professional medical services,-0.000001,5.019934e-08,8.101768e-07,-1.304367e-07,0.000541,4.629929e-08,-3.546950e-07,-3.020354e-07,1.697564e-06,1.762362e-08,...,1.500771e-07,0.000013,-1.677389e-07,-4.301805e-08,-1.135382e-07,0.000002,-0.000001,4.763465e-07,-2.937917e-07,2.403356e-07
"Amusement parks, campgrounds, and related recreational services",-0.000001,2.828957e-08,5.977306e-07,-9.726589e-08,0.000064,4.309732e-08,-4.443453e-07,-2.229691e-07,8.416852e-07,3.266434e-09,...,1.029091e-07,0.000012,-1.539774e-07,-2.588297e-08,-1.210438e-07,0.000004,-0.000002,5.018007e-07,-2.266724e-07,2.136081e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Water supply and sewage maintenance,-0.000001,4.048809e-08,7.963240e-07,-1.092269e-07,0.001195,3.431087e-08,-2.606318e-07,-3.758518e-07,2.195219e-06,2.379172e-08,...,1.773838e-07,0.000014,-1.642300e-07,-6.302634e-08,-8.711866e-08,-0.000004,-0.000001,4.171205e-07,-2.959626e-07,2.380489e-07
Water transportation,-0.000001,3.740786e-08,6.327216e-07,-8.952642e-08,0.001250,5.205089e-08,-2.589892e-07,-2.541221e-07,1.528410e-06,1.466534e-08,...,1.414846e-07,0.000013,-9.795894e-08,-2.154867e-08,-7.762926e-08,0.000004,-0.000001,3.822507e-07,-1.627652e-07,1.910580e-07
Window coverings,0.000008,-3.256220e-07,-1.256764e-06,-3.123718e-07,-0.006681,2.581891e-07,-3.500204e-06,3.589651e-06,-1.108246e-05,-2.121848e-07,...,-8.022229e-07,-0.000007,4.670728e-08,6.847659e-07,-1.381788e-07,0.000041,0.000004,5.937884e-06,-4.668907e-07,-9.727748e-07


In [82]:
omega = delta_products_saleshare.T

In [83]:
omega

key_0,Accessories and parts,Air transportation,Alcohol in purchased meals,All other professional medical services,"Amusement parks, campgrounds, and related recreational services","Audio discs, tapes, vinyl, and permanent digital downloads",Audio equipment,Auto leasing,Bakery products,Beef and veal,...,Vegetables (fresh),Veterinary and other services for pets,Video and audio streaming and rental,"Video discs, tapes, and permanent digital downloads",Watches,Water supply and sewage maintenance,Water transportation,Window coverings,Wine,Women's and girls' clothing
Accessories and parts,-1.248365e-06,-1.684369e-06,-1.255984e-06,-1.384358e-06,-1.270555e-06,-1.592079e-05,-1.886157e-06,-1.392788e-06,-9.529391e-07,-7.513544e-06,...,-1.288388e-06,-1.167090e-06,-1.275082e-06,-1.541359e-05,-1.490454e-06,-1.362507e-06,-1.120545e-06,7.501430e-06,-2.486117e-06,-1.684256e-06
Air transportation,2.153676e-07,1.511755e-08,2.686052e-08,5.019934e-08,2.828957e-08,6.891520e-07,5.700827e-08,-2.288144e-08,3.138625e-08,3.665573e-07,...,3.132258e-08,3.735722e-08,-3.099686e-09,5.226435e-07,6.340320e-08,4.048809e-08,3.740786e-08,-3.256220e-07,1.338465e-07,4.398719e-08
Alcohol in purchased meals,1.660210e-06,4.982699e-07,6.984471e-07,8.101768e-07,5.977306e-07,7.079331e-07,8.237410e-07,2.809233e-07,7.672782e-07,2.559040e-06,...,5.927706e-07,7.796161e-07,3.256519e-07,3.216611e-06,9.031301e-07,7.963240e-07,6.327216e-07,-1.256764e-06,1.391766e-06,7.289361e-07
All other professional medical services,-8.503305e-08,-7.560467e-08,-1.323518e-07,-1.304367e-07,-9.726589e-08,3.863606e-07,-1.563661e-07,-1.208297e-07,-2.276662e-07,1.377843e-07,...,-1.220355e-07,-1.530502e-07,-4.424245e-08,-5.258056e-06,-1.368577e-07,-1.092269e-07,-8.952642e-08,-3.123718e-07,-7.231434e-08,-1.691446e-07
"Amusement parks, campgrounds, and related recreational services",4.259495e-03,1.527363e-04,1.438971e-03,5.405775e-04,6.351994e-05,2.454073e-02,-1.227335e-04,9.378186e-04,7.878854e-04,-1.985604e-02,...,1.544942e-04,-7.954892e-04,1.382279e-03,-8.611394e-02,1.211432e-03,1.195492e-03,1.250288e-03,-6.681042e-03,1.150419e-03,3.153765e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Water supply and sewage maintenance,-2.645616e-05,3.455617e-06,2.263512e-06,1.833065e-06,4.429217e-06,-7.556090e-05,6.857456e-06,5.625569e-06,-2.503243e-06,-6.676977e-05,...,1.078112e-06,8.750677e-06,9.207786e-07,-4.751235e-05,-2.157404e-06,-3.945061e-06,4.377763e-06,4.126506e-05,-7.079576e-06,-1.089152e-06
Water transportation,-2.730320e-06,-1.582842e-06,-1.091780e-06,-1.270450e-06,-1.534713e-06,-1.140213e-05,-1.673003e-06,-1.300085e-06,-1.340344e-06,-6.872244e-06,...,-1.756095e-06,-1.593849e-06,-1.020647e-06,-5.600762e-05,-1.354077e-06,-1.045344e-06,-1.087362e-06,4.079587e-06,-1.119865e-06,-2.149014e-06
Window coverings,2.241393e-07,4.628306e-07,3.162558e-07,4.763465e-07,5.018007e-07,-2.648762e-06,5.024647e-07,1.380735e-07,2.449249e-07,4.061162e-06,...,4.923001e-07,7.371898e-07,2.179538e-07,2.297489e-05,4.663168e-07,4.171205e-07,3.822507e-07,5.937884e-06,4.206249e-07,6.408713e-07
Wine,-6.383776e-07,-1.948936e-07,-1.697132e-07,-2.937917e-07,-2.266724e-07,-1.028187e-07,-1.717010e-07,-2.104581e-08,-3.885144e-07,-3.057658e-06,...,-2.816586e-07,-2.666447e-07,-1.686757e-07,-6.773446e-06,-3.862408e-07,-2.959626e-07,-1.627652e-07,-4.668907e-07,6.300953e-07,-3.203439e-07
